In [5]:
pip install keras


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\admin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
import json
import numpy as np
#import tensorflow.keras.preprocessing.sequence
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
#import keras.preprocessing.sequence.pad_sequences


In [7]:
# Load JSON file containing captions
with open('D:\\Major23\\image_frames1.json', 'r') as f:
    captions = json.load(f)

# Extract text and frames from captions
captions_text = [caption['caption'] for caption in captions]
captions_frame_paths = [caption['folder_path'] for caption in captions]

# Tokenize captions text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions_text)
captions_sequences = tokenizer.texts_to_sequences(captions_text)


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Major23\\image_frames1.json'

In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [5]:


# Tokenize captions text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions_text)
captions_sequences = tokenizer.texts_to_sequences(captions_text)

# Pad sequences to a fixed length
max_sequence_length = max(len(seq) for seq in captions_sequences)
padded_captions_sequences = pad_sequences(captions_sequences, maxlen=max_sequence_length)




In [6]:

# Define LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_sequence_length))
model.add(LSTM(units=128))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [7]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(padded_captions_sequences, np.zeros(len(captions)), epochs=5, batch_size=32)

Epoch 1/5
2/2 [==============================] - 2s 16ms/step - loss: 2.8540e-04
Epoch 2/5
2/2 [==============================] - 0s 18ms/step - loss: 6.5941e-05
Epoch 3/5
2/2 [==============================] - 0s 14ms/step - loss: 7.8428e-05
Epoch 4/5
2/2 [==============================] - 0s 16ms/step - loss: 9.2517e-06
Epoch 5/5
2/2 [==============================] - 0s 21ms/step - loss: 3.3127e-05


In [8]:
model.save('textRetr.h5')

In [9]:
# Define a query
query = "a man in a band playing a guitar"

# Tokenize and pad the query
query_sequence = tokenizer.texts_to_sequences([query])
padded_query_sequence = pad_sequences(query_sequence, maxlen=max_sequence_length)

In [10]:
import os

In [11]:
# Use the model to predict the caption with the highest similarity score
similarity_scores = model.predict(padded_captions_sequences)
max_index = np.argmax(similarity_scores)

start_index = max_index + 1
end_index = min(start_index + 20, len(captions))
relevant_frame_paths = [frame_path for frames in captions_frame_paths[start_index:end_index] for frame_path in frames]

# Construct file paths for relevant frames
frame_dir = 'D:\\Major23\\Filtered'
relevant_frame_file_paths = [os.path.join(frame_dir, frame_path) for frame_path in relevant_frame_paths]


2/2 [==============================] - 0s 10ms/step


In [12]:
import json

# Specify the path of the JSON file
json_file_path = 'D:\\Major23\\relevant_frames_t_t_c_copy.json'

# Create a dictionary to store the relevant frame file paths
data = {'relevant_frames': relevant_frame_file_paths}

# Write the data to the JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file)


In [23]:
import os

In [13]:
print(relevant_frame_file_paths)


['D:\\Major23\\Filtered\\D', 'D:\\Major23\\Filtered\\:', 'D:\\', 'D:\\Major23\\Filtered\\M', 'D:\\Major23\\Filtered\\a', 'D:\\Major23\\Filtered\\j', 'D:\\Major23\\Filtered\\o', 'D:\\Major23\\Filtered\\r', 'D:\\Major23\\Filtered\\2', 'D:\\Major23\\Filtered\\3', 'D:\\', 'D:\\Major23\\Filtered\\F', 'D:\\Major23\\Filtered\\i', 'D:\\Major23\\Filtered\\l', 'D:\\Major23\\Filtered\\t', 'D:\\Major23\\Filtered\\e', 'D:\\Major23\\Filtered\\r', 'D:\\Major23\\Filtered\\e', 'D:\\Major23\\Filtered\\d', 'D:\\Major23\\Filtered\\D', 'D:\\Major23\\Filtered\\:', 'D:\\', 'D:\\Major23\\Filtered\\M', 'D:\\Major23\\Filtered\\a', 'D:\\Major23\\Filtered\\j', 'D:\\Major23\\Filtered\\o', 'D:\\Major23\\Filtered\\r', 'D:\\Major23\\Filtered\\2', 'D:\\Major23\\Filtered\\3', 'D:\\', 'D:\\Major23\\Filtered\\F', 'D:\\Major23\\Filtered\\i', 'D:\\Major23\\Filtered\\l', 'D:\\Major23\\Filtered\\t', 'D:\\Major23\\Filtered\\e', 'D:\\Major23\\Filtered\\r', 'D:\\Major23\\Filtered\\e', 'D:\\Major23\\Filtered\\d', 'D:\\Major23\\F

In [20]:
import shutil

# Specify the output folder path
result = 'D:\\Major23\\RelevantFrames_copy'

# Create the output folder if it doesn't exist
if not os.path.exists(result):
    os.makedirs(result)

# Copy the relevant frames to the output folder
for frame_path in relevant_frame_file_paths:
    try:
        # Extract the filename from the frame path
        filename = os.path.basename(frame_path)

        # Construct the output file path
        output_file_path = os.path.join(result, filename)

        # Copy the frame to the output folder
        shutil.copy2(frame_path, output_file_path)
        print(f"Successfully copied {frame_path} to {output_file_path}")
    except FileNotFoundError:
        print(f"Error: File {frame_path} not found.")

print("All relevant frames have been copied to the output folder.")


Error: File D:\Major23\Filtered\D not found.


OSError: [Errno 22] Invalid argument: 'D:\\Major23\\Filtered\\:'

In [28]:
import shutil
import cv2

# Specify the output folder path
output_folder_path = 'D:\\Major23\\RelevantFrames_copy'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Read the frames from the output.json file
with open('relevant_frames_t_t_c_copy.json', 'r') as json_file:
    frame_data = json.load(json_file)

# Copy the relevant frames to the output folder
for frame_info in frame_data:
    try:
        # Get the frame path from the JSON data
        frame_path = frame_info['frame_path']

        # Read the frame from the file
        frame = cv2.imread(frame_path)

        # Check if the frame is successfully read
        if frame is None:
            print(f"Error: Failed to read frame from {frame_path}")
            continue

        # Extract the filename from the frame path
        filename = os.path.basename(frame_path)

        # Construct the output file path
        output_file_path = os.path.join(output_folder_path, filename)

        # Copy the frame to the output folder
        cv2.imwrite(output_file_path, frame)
        print(f"Successfully copied {frame_path} to {output_file_path}")
    except FileNotFoundError:
        print(f"Error: File {frame_path} not found.")

print("All relevant frames have been copied to the output folder.")


TypeError: string indices must be integers, not 'str'

In [23]:
import json

def convert_frames_to_video(input_list, output_file_name, fps, size):
    # Define the output video writer object 
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Define the codec (FourCC code)
    out = cv2.VideoWriter(output_file_name, fourcc, fps, size)
    
    # Loop through the input frames and add them to the output video
    for i, img_name in enumerate(input_list):
        img_path = os.path.join(input_frame_path, img_name)
        img = cv2.imread(img_path)
        
        if img is None: # Skip if the image cannot be read
            print(img_name + ' does not exist')
            continue
        
        out.write(img) # Write out frame to video
        
        # Show the image in a window (optional)
        cv2.imshow('img', img)
        cv2.waitKey(1)
    
    # Release everything if job is finished
    out.release()
    cv2.destroyAllWindows()
    
    print("The output video {} is saved.".format(output_file_name))

if __name__=='__main__':
    # Set up paths and directories
    data_dir = 'D:\\Major23'
    data_subdir = 'Filtered'
    input_frame_path = os.path.join(data_dir, data_subdir)
    
    output_vid_dir = 'D:\\Major23\\video'
    if not os.path.exists(output_vid_dir):
        os.mkdir(output_vid_dir)
    
    # Set up parameters for output video
    fps = 5
    output_file_name = os.path.join(output_vid_dir, 'o/p_video1.mp4')
    
    # Load the relevant frame file paths from JSON
    json_file_path = 'D:\\Major23\\relevant_frames_t_t_c_copy.json'
    if os.path.exists(json_file_path):
        with open(json_file_path) as json_file:
            json_data = json.load(json_file)
        
        relevant_frame_file_paths = json_data.get('relevant_frames', [])
    else:
        print("JSON file does not exist.")
        relevant_frame_file_paths = []
    
    # Get the size of the frames from the first image
    if len(relevant_frame_file_paths) > 0:
        frame = cv2.imread(relevant_frame_file_paths[0])
        height, width, channels = frame.shape
        size = (width, height)
        
        # Convert the frames to a video
        convert_frames_to_video(relevant_frame_file_paths, output_file_name, fps, size)
    else:
        print("No relevant frame file paths found.")


AttributeError: 'NoneType' object has no attribute 'shape'

In [22]:
import cv2

In [18]:
import cv2
import os

def convert_frames_to_video(input_list, output_file_name, fps, size):
    # Define the output video writer object 
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Define the codec (FourCC code)
    out = cv2.VideoWriter(output_file_name, fourcc, fps, size)
    
    # Loop through the input frames and add them to the output video
    for i, img_name in enumerate(input_list):
        img_path = os.path.join(input_frame_path, img_name)
        img = cv2.imread(img_path)
        
        if img is None: # Skip if the image cannot be read
            print(img_name + ' does not exist')
            continue
        
        out.write(img) # Write out frame to video
        
        # Show the image in a window (optional)
        cv2.imshow('img', img)
        cv2.waitKey(1)
    
    # Release everything if job is finished
    out.release()
    cv2.destroyAllWindows()
    
    print("The output video {} is saved.".format(output_file_name))

if __name__=='__main__':
    # Set up paths and directories
    data_dir = 'D:\\Major23'
    data_subdir = 'Filtered'
    input_frame_path = os.path.join(data_dir, data_subdir)
    
    output_vid_dir = 'D:\\Major23\\video'
    if not os.path.exists(output_vid_dir):
        os.mkdir(output_vid_dir)
    
    # Set up parameters for output video
    fps = 5
    output_file_name = os.path.join(output_vid_dir, 'outputvideo1.mp4')
    
    # Get the size of the frames from the first image
    frame = cv2.imread(os.path.join(input_frame_path, os.listdir(input_frame_path)[0]))
    height, width, channels = frame.shape
    size = (width, height)
    
    # Convert the frames to a video
    img_list = sorted(os.listdir(input_frame_path)) # Sort the image names
    convert_frames_to_video(img_list, output_file_name, fps, size)


The output video D:\Major23\video\outputvideo21.mp4 is saved.


In [21]:
import cv2
import os

# Set up video writer
video_dir = 'D:\\Major23\\video'
if not os.path.exists(video_dir):
    os.makedirs(video_dir)
video_path = os.path.join(video_dir, 'output_video.mp4')
frame_width = 640  # Adjust to desired frame size
frame_height = 480
fps = 30
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_path, fourcc, fps, (frame_width, frame_height))

# Iterate over frames and add to video
frame_dir = 'D:\\Major23\\Filtered'  # Specify the directory where the frames are located
for frame_path in relevant_frame_file_paths:
    # Load frame
    frame = cv2.imread(frame_path)

    # Check if frame is valid
    if frame is not None:
        # Resize frame to desired dimensions
        frame = cv2.resize(frame, (frame_width, frame_height))

        # Write frame to video
        video_writer.write(frame)

# Release video writer
video_writer.release()
